In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(
        update={
            "favourite_colour": favourite_colour,
            "messages": [
                ToolMessage(
                    "Successfully updated favourite colour",
                    tool_call_id=runtime.tool_call_id,
                )
            ],
        }
    )

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState,
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [6]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='a0327851-0d2e-4228-9f5e-30f50eeef878'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 411, 'prompt_tokens': 141, 'total_tokens': 552, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 384, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqmAtktNDqyT86PjXxB9Gb0bOGXNz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b572f-5996-7040-b1e4-5cb8cc78ea54-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_hiKJQBqfIjh8Stwc31dHpvhI', 'type': 'tool_call'}], usage_me

In [7]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='0c6bfff1-d954-4e76-8e97-308c697f32f1'),
              AIMessage(content="I'm doing well, thanks for asking! How can I help you today? If you have a question, need help with something, or just want to chat, I'm here.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 236, 'prompt_tokens': 142, 'total_tokens': 378, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqmTc294N9Q6yprzfm0bfaLxqnJJ3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b5741-0f16-79e3-94b8-3504c2f8bc9c-0', usage_metadata={'input_tokens'

## Read state

In [8]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [9]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='4ceaae29-ae59-4e8d-85e1-f4f88bdfa972'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 539, 'prompt_tokens': 162, 'total_tokens': 701, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqmUebPvi9CSOA5GLXrKgBdI7OZrc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5742-05d1-7e82-adcc-e8b0e2afd763-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_zZgv0wA5z6ibt22OBf4YBrsF', 'type': 'tool_call'}], usage_me

In [10]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='4ceaae29-ae59-4e8d-85e1-f4f88bdfa972'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 539, 'prompt_tokens': 162, 'total_tokens': 701, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqmUebPvi9CSOA5GLXrKgBdI7OZrc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5742-05d1-7e82-adcc-e8b0e2afd763-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'call_zZgv0wA5z6ibt22OBf4YBrsF', 'type': 'tool_call'}], usage_me